**Notebook preparation and importing libraries**

In [1]:
import warnings
from hvac import Quantity
from hvac.fluids import HumidAir, Fluid, CoolPropWarning

In [2]:
warnings.filterwarnings('ignore', category=CoolPropWarning)

In [3]:
Q_ = Quantity

In [4]:
from hvac.heat_transfer.heat_exchanger.fin_tube import PlainFinTubeCounterFlowAirCondenser as Condenser

# Rating of a plain fin-tube counterflow air condenser

Define the type of refrigerant:

In [5]:
R134a = Fluid('R134a')

Define the type of condenser: a plain fin-tube, counterflow heat exchanger.

Create an instance of the condenser model, specifying the dimensions of the frontal area, the number of rows, the geometrical characteristics of the heat transfer surfaces, and the thermal conductivity of the fins (by default aluminum fins are assumed):

In [6]:
condenser = Condenser(
    W_fro=Q_(0.909, 'm'),
    H_fro=Q_(0.303, 'm'),
    N_rows=4,
    S_trv=Q_(25.4, 'mm'),         # vertical distance between tubes
    S_lon=Q_(22.0, 'mm'),         # horizontal distance between tubes
    D_int=Q_(8.422, 'mm'),        # inner tube diameter
    D_ext=Q_(10.2, 'mm'),         # outer tube diameter
    t_fin=Q_(0.3302, 'mm'),       # fin thickness
    N_fin=1 / Q_(3.175, 'mm'),    # fin density
    k_fin=Q_(237, 'W / (m * K)')  # conductivity of fin material
)

On this object, we set the known operating conditions:
- the mass flow rate of air through the condenser
- the mass flow rate of refrigerant, which was determined when rating the evaporator
- the state of air that enters the condenser
- the state of refrigerant entering the condenser

The result of the condenser rating will contain:
- the state of refrigerant leaving the condenser
- the state of air leaving the condenser
- the heat rejection rate (the heat transfer rate from the refrigerant to the air stream)
- the degree of subcooling
- the flow length of air through the desuperheating, condensing, and subcooling part of the condenser

The rating routine (of the subcooling part) is an iterative routine. The criterion to stop the iterative routine is the tolerable deviation between the last and the previous calculated heat exchanger effectiveness, which can be set through parameter `tol_eps`. Should the tolerable deviation not be reached after a maximum number of iterations, which can be set with parameter `i_max`, a `ValueError` exception will be raised and no result is returned.

In [7]:
air_out, rfg_out = condenser.solve(
    air_in=HumidAir(Tdb=Q_(35.0, 'degC'), RH=Q_(30, 'pct')),
    air_m_dot=Q_(2233.861, 'kg / hr'),
    rfg_in=R134a(T=Q_(85.0, 'degC'), P=Q_(13.179, 'bar')),
    rfg_m_dot=Q_(99.908, 'kg / hr')
)

In [8]:
print(
    f"air out = {air_out.Tdb.to('degC'):~P.1f} DB, "
    f"{air_out.RH.to('pct'):~P.0f} RH\n"
    f"refrigerant out = {rfg_out.T.to('degC'):~P.1f}\n"
    f"degree of subcooling = {condenser.dT_sc.to('K'):~P.2f}\n"
    f"heat rejection rate = {condenser.Q_dot.to('kW'):~P.3f}\n"
    f"L2 desuperheating = {condenser.desuperheating_region.L_flow.to('mm'):~P.0f}\n"
    f"L2 condensing = {condenser.condensing_region.L_flow.to('mm'):~P.0f}\n"
    f"L2 subcooling = {condenser.subcooling_region.L_flow.to('mm'):~P.0f}\n"
    f"air-side pressure drop = {condenser.air_dP.to('Pa'):~P.3f}"
)

air out = 43.9 °C DB, 19 % RH
refrigerant out = 42.3 °C
degree of subcooling = 7.75 K
heat rejection rate = 5.647 kW
L2 desuperheating = 26 mm
L2 condensing = 53 mm
L2 subcooling = 9 mm
air-side pressure drop = 31.848 Pa
